In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import copy

In [46]:
def sign(x):
    if x > 0 :
        return 1
    else:
        return -1

In [47]:
# 软间隔支持向量机的SMO实现
class SoftMarginSVM(object):
    #初始化各项参数
    def __init__(self, epochs=100, C=1.0):
        self.w = None
        self.b = None
        self.alpha = None
        self.E = None
        self.epochs = epochs
        self.C = C
        # 记录支持向量
        self.support_vectors = None

    def init_params(self, X, y):
        n_samples, n_features = X.shape   # 记录样本数和特征数
        self.w = np.zeros(n_features)     # 初始化权值矩阵
        self.b = 0
        self.alpha = np.zeros(n_samples)  # 初始化ai
        self.E = np.zeros(n_samples)      # 初始化误差
        for i in range(0, n_samples):
            self.E[i] = np.dot(self.w, X[i, :]) + self.b - y[i]
    # SMO中固定aj的值
    def select_j(self, best_i):
        # 设定可以取到的j的合理范围
        valid_j_list = [i for i in range(0, len(self.alpha)) if self.alpha[i] > 0 and i != best_i]
        best_j = -1
        # 优先选择使得|Ei-Ej|最大的j，即变化最大的点
        if len(valid_j_list) > 0:
            max_e = 0
            for j in valid_j_list:
                current_e = np.abs(self.E[best_i] - self.E[j])
                if current_e > max_e:
                    best_j = j
                    max_e = current_e
        # 否则随机选择
        else:
            list1 = list(range(len(self.alpha)))
            seq = list1[: best_i] + list1[best_i + 1:]
            best_j = random.choice(seq)
        return best_j
    # 判定各点是否满足kkt条件
    def meet_kkt(self, w, b, x_i, y_i, alpha_i):
        if alpha_i < self.C:
            return y_i * (np.dot(w, x_i) + b) >= 1
        else:
            return y_i * (np.dot(w, x_i) + b) <= 1

    def fit(self, X, y2):
        y = copy.deepcopy(y2)
        y[y == 0] = -1
        self.init_params(X, y)
        for _ in range(0, self.epochs):        # 不断迭代直到所有点满足kkt条件
            if_all_match_kkt = True
            # 搜寻违反KKT条件的点i，并更新它的alpha值
            for i in range(0, len(self.alpha)):
                x_i = X[i, :]
                y_i = y[i]
                alpha_i_old = self.alpha[i]
                E_i_old = self.E[i]
                if not self.meet_kkt(self.w, self.b, x_i, y_i, alpha_i_old):     # 如果不满足kkt条件，则固定该点对应的alpha
                    if_all_match_kkt = False
                    best_j = self.select_j(i)                                    # 在该基础上固定alpha_j
                    alpha_j_old = self.alpha[best_j]
                    x_j = X[best_j, :]
                    y_j = y[best_j]
                    E_j_old = self.E[best_j]
            # 对选好的两个alpha进行更新
            # 首先获取无裁剪的最优alpha_j
            eta = np.dot(x_i - x_j, x_i - x_j)
            # 如果x_i和x_j很接近，则跳过
            if eta < 1e-3:
                continue
            alpha_j_unc = alpha_j_old + y_j * (E_i_old - E_j_old) / eta
            # 裁剪并得到new alpha_j
            if y_i == y_j:
                L = max(0., alpha_i_old + alpha_j_old - self.C)
                H = min(self.C, alpha_i_old + alpha_j_old)
            else:
                L = max(0, alpha_j_old - alpha_i_old)
                H = min(self.C, self.C + alpha_j_old - alpha_i_old)
            if alpha_j_unc < L:
                alpha_j_new = L
            elif alpha_j_unc > H:
                alpha_j_new = H
            else:
                alpha_j_new = alpha_j_unc
            # 如果变化不够大则跳过
            if np.abs(alpha_j_new - alpha_j_old) < 1e-5:
                continue
            # 得到alpha_i_new
            alpha_i_new = alpha_i_old + y_i * y_j * (alpha_j_old - alpha_j_new)
            # 更新w
            self.w = self.w + (alpha_i_new - alpha_i_old) * y_i * x_i + (alpha_j_new - alpha_j_old) * y_j * x_j
            # 更新alpha_i,alpha_j
            self.alpha[i] = alpha_i_new
            self.alpha[best_j] = alpha_j_new
            # 更新b
            b_i_new = y_i - np.dot(self.w, x_i)
            b_j_new = y_j - np.dot(self.w, x_j)
            if self.C > alpha_i_new > 0:
                self.b = b_i_new
            elif self.C > alpha_j_new > 0:
                self.b = b_j_new
            else:
                self.b = (b_i_new + b_j_new) / 2.0
            # 更新E
            for k in range(0, len(self.E)):
                self.E[k] = np.dot(self.w, X[k, :]) + self.b - y[k]
            # 如果所有的点都满足KKT条件，则中止
            if if_all_match_kkt is True:
                break
        # 计算支持向量epochs=100
        self.support_vectors = np.where(self.alpha > 1e-3)[0]
    # 显示最终结果
    def get_params(self):
        return self.w, self.b
    # 预测函数
    def predict(self, x):
        return x.dot(self.w) + self.b

In [48]:


if __name__== "__main__":
    # 训练集
    #usecols=['area', 'perimeter', 'compactness',' length', 'width', 'asymmetry coefficient', 'length of kernel groove']
    data = pd.read_csv('seeds_dataset.csv', header=None, sep=',' )
    data = np.array(data)
    np.random.seed(5)
    #随机打乱样本
    np.random.shuffle(data)
    data_test1 =data[84:, :]#选取测试样本
    data_test2 =data[:56, :]#选取测试样本
    data_train = data[56:84, :] #选取训练样本
    data_test = np.concatenate((data_test1,data_test2),axis=0) #选取测试样本
    #print(data_test)
    X = data_train[:, :7]
    Y = data_train[:, 7:]
    #print(Y)
    X_test = data_test[:, :7]
    Y_test = data_test[:, 7:]
    #test =np.linspace(0.01,10,100)
    #accuracy_total = []
    #print(test)
    #for k in test:
    count = 0
    svm = SoftMarginSVM(epochs=100, C=0.1)
    svm.fit(X, Y)
    Z_test = svm.predict(X_test)
    #print(Z_test)
    #print(Y_test)
    # 若预测结果与真实结果相差不到0.5，则认为分类正确
    for i in range(Y_test.shape[0]):
        Z_test[i] = sign(Z_test[i])
        if (Z_test[i] == Y_test[i]):
            count += 1
    accuracy = count / Y_test.shape[0]
    #accuracy_total.append(accuracy)
    print(accuracy)     #输出准确率
    #plt.plot(test, accuracy_total)
    #plt.xlabel('C')
    #plt.ylabel('Accuracy')
    

0.9099099099099099
